In [2]:
import sys
!{sys.executable} -m pip install keras
!{sys.executable} -m pip install opencv-python
!pip install tensorflowjs
!pip install matplotlib

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


Defaulting to user installation because normal site-packages is not writeable


In [10]:
import numpy as np
import os
from matplotlib import pyplot as plt
import cv2
import random
import pickle


file_list = []
class_list = []

DATADIR = "data"

# All the categories you want your neural network to detect
CATEGORIES = ["cat", "dog"]

# The size of the images that your neural network will use
IMG_SIZE = 30

# Checking or all images in the data folder
for category in CATEGORIES :
    path = os.path.join(DATADIR, category)
    for img in os.listdir(path):
        img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)

training_data = []

def create_training_data():
    for category in CATEGORIES :
        path = os.path.join(DATADIR, category)
        class_num = CATEGORIES.index(category)
        for img in os.listdir(path):
            try :
                img_array = cv2.imread(os.path.join(path, img), cv2.IMREAD_GRAYSCALE)
                new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
                training_data.append([new_array, class_num])
            except Exception as e:
                pass

create_training_data()

random.shuffle(training_data)

X = [] #features
y = [] #labels

for features, label in training_data:
    X.append(features)
    y.append(label)

X = np.array(X).reshape(-1, IMG_SIZE, IMG_SIZE, 1)

# Creating the files containing all the information about your model
pickle_out = open("X.pickle", "wb")
pickle.dump(X, pickle_out)
pickle_out.close()

pickle_out = open("y.pickle", "wb")
pickle.dump(y, pickle_out)
pickle_out.close()

pickle_in = open("X.pickle", "rb")
X = pickle.load(pickle_in)

In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
from keras.models import model_from_json
from keras.models import load_model
import matplotlib.pyplot as plt

# Opening the files about data
X = pickle.load(open("X.pickle", "rb"))
y = pickle.load(open("y.pickle", "rb"))
y = np.array(y) #https://github.com/tensorflow/tensorflow/issues/38613

# normalizing data (a pixel goes from 0 to 255)
X = X/255.0

callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

# Building the model
model = Sequential()
# 3 convolutional layers
model.add(Conv2D(32, (3, 3), input_shape = X.shape[1:]))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation("relu"))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

# hidden layers
model.add(Flatten())

model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))

model.add(Dense(128))
model.add(Activation("relu"))

# The output layer with 2 neurons, for 2 classes
model.add(Dense(2))
model.add(Activation("softmax"))

# Compiling the model using some basic parameters
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

# Training the model
# validation_split corresponds to the percentage of images used for the validation phase compared to all the images
history = model.fit(X, y, batch_size=32, epochs=20, validation_split=0.1, callbacks=[callback])

# Saving the model
model_json = model.to_json()
with open("model.json", "w") as json_file :
    json_file.write(model_json)

# python
model.save_weights("model.h5")
model.save('CNN.model')

#javascript
saved_model_path = "./model.js.h5"
model.save(saved_model_path)
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

print("Saved model to disk")

Epoch 1/20
63/63 [==============================] - 2s 30ms/step - loss: 0.6949 - accuracy: 0.4953 - val_loss: 0.6933 - val_accuracy: 0.4664
Epoch 2/20
63/63 [==============================] - 2s 27ms/step - loss: 0.6934 - accuracy: 0.5022 - val_loss: 0.6939 - val_accuracy: 0.4664
Epoch 3/20
63/63 [==============================] - 2s 27ms/step - loss: 0.6932 - accuracy: 0.5107 - val_loss: 0.6914 - val_accuracy: 0.5381
Epoch 4/20
63/63 [==============================] - 2s 28ms/step - loss: 0.6910 - accuracy: 0.5197 - val_loss: 0.6901 - val_accuracy: 0.5336
Epoch 5/20
63/63 [==============================] - 2s 29ms/step - loss: 0.6928 - accuracy: 0.5052 - val_loss: 0.6877 - val_accuracy: 0.5381
Epoch 6/20
63/63 [==============================] - 2s 30ms/step - loss: 0.6845 - accuracy: 0.5527 - val_loss: 0.6801 - val_accuracy: 0.5695
Epoch 7/20
63/63 [==============================] - 2s 28ms/step - loss: 0.6741 - accuracy: 0.5682 - val_loss: 0.6516 - val_accuracy: 0.6413
Epoch 8/20
63

In [7]:
model = tf.keras.models.load_model("CNN.model")

NameError: name 'tf' is not defined

In [6]:
CATEGORIES = ["cat", "dog"]

def prepare(file):
    IMG_SIZE = 30
    img_array = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
    new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
    return new_array.reshape(-1, IMG_SIZE, IMG_SIZE, 1)

image = prepare("./samples/cat.123.jpg")
prediction = model.predict([image])
prediction = list(prediction[0])
print(prediction)
print(CATEGORIES[prediction.index(max(prediction))])


NameError: name 'model' is not defined

In [32]:
import json
import requests
import cv2
import numpy as np

class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

IMG_SIZE = 30

img_array = cv2.imread("./samples/cat.125.jpg", cv2.IMREAD_GRAYSCALE)
new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
new_array1 = new_array.flatten()
s1 = json.dumps(new_array1, cls=NumpyEncoder)
url = 'http://localhost:3000/analyze-image'
x = requests.post(url, headers = {'Content-Type': 'application/json'}, data=s1)
print(x.text)

dog
